Parametrit
* $i \in \{1, \dots, n\}$, geenin indeksi (gene index)
* $j \in \{1, \dots, m\}$, näytteen indeksi (sample index)
* $r \in \{0, \dots, p-1\}$, kovarianssiindeksi (covariate index, with intercept r = 0)
* $K_{ij}$ geenin $i$ sekvenssilukumäärä näytteesä $j$ (counts of reads for gene i, sample j)
* $\mu_{ij}$ sovitettu keskiarvo (fitted mean)
* $\alpha_i$ geenikohtainen hajonta (gene-specific dispersion)
* $s_j$ näytekohtainen suuruustekijä (sample-specific size factor)
* $s_{ij}$ geeni- ja näytekohtainen normalisaatiovakio (gene- and sample-specific normalization factor)
* $q_{ij}$ proportional to true concentration of fragments
* $x_{jr}$ elements of the design matrix $X$
*

$$
NB(y | \mu, \alpha) = {y + \alpha -1 \choose y}(\frac{\mu}{\mu + \alpha})^n (\frac{\alpha}{\mu + \alpha})^\alpha
$$

Sekvenssilukumäärä $K_{ij}$ geenille $i$ näytteessä $j$ mallinnetaan negatiivisen binomijakauman yleistetyllä lineaarisella mallilla:

$$
K_{ij} \sim NB(\mu_{ij}, \alpha_i)
$$

$$
\mu_{ij} = s_{ij}q_{ij}
$$

$$
\log q_{ij} = \sum_r x_{jr} \beta_{ir}
$$

Oletusarvoisesti normalisaatiovakiot $s_{ij}$ ovat vakioita yhdessä näytteessä, eli $s_{ij} = s_j$. Normalisaatiovakiot estimoidaan osamäärien mediaanimenetelmällä:

$$
s_j = \textrm{median}\frac{K_{ij}}{K_i^R}, K_i^R \neq 0 
$$
jossa
$$
K_i^R = (\prod_{j=1}^m K_{ij})^{1/m}
$$

### Expanded design matrices

Lineaarisessa mallinnuksessa luokkamuuttuja tai tekijä voi saada kaksi tai useampaa arvoa tai tasoa. Yleensä sunnittelumatriisissa yksi luokka on viitearvo (base level) ja se liitetään interceptiin. Standardi GLM:ssä viitetason valinta ei vaikuta kontrasteihin (LFCs). Tämä ei kuitenkaan päde tässä, kun käytetään ridge-regression-like shrinkage vakioolle, kun luokkamuuttjalla on enemmän kuin kaksi luokkaa. Tämä johtuu siitä, että viitearvoon ei kohdistu shrinkage, mutta muihin tasoihin kyllä.  

### Contrasts